# Лабараторая работа № 2

## Описание датасета

**предметная область**: шахматные фигуры

**источник датасета**: [https://www.kaggle.com/datasets/niteshfre/chessman-image-dataset](https://www.kaggle.com/datasets/niteshfre/chessman-image-dataset)

**характер данных**: реальный

Предварительно разделим датасет на 2 папки - тренировочные(80%) и тесвоые(20%)

In [1]:
pip install numpy pandas matplotlib tensorflow keras opencv-python

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 2.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/11.6 MB 2.9 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.6 MB 3.2 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/11.6 MB 3.5 MB/s eta 0:00:03
   -------------- ------------------------- 4.2/11.6 MB 3.9 MB/s eta 0:00:02
   ------------------ --------------------- 5.5/11.6 MB 4.2 MB/s eta 0:00:02
   ------------------------ --------------- 7.1/11.6 MB 4.7 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.6 MB 5.2 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.6 MB 5.9 MB/s eta 0:00:01
   ----------------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 3.9 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.1 MB 3.7 MB/s eta 0:00:03
   ---------- ----------------------------- 2.9/11.1 MB 4.0 MB/s eta 0:00:03
   ------------- -------------------------- 3.7/11.1 MB 4.1 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.1 MB 4.4 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.1 MB 4.6 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.1 MB 4.3 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.1 MB 4.3 MB/s eta 0:00:01
   -------------------------------- ------- 9.2/11.1 MB 4.6 MB/s eta 0:00:01
   --------------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

Настройка констант

In [12]:
IMG_SIZE = 224  # Размер изображений для EfficientNet
BATCH_SIZE = 32
EPOCHS = 10


train_dir = "data/train"
val_dir = "data/val"

Передача данных

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 436 images belonging to 6 classes.
Found 115 images belonging to 6 classes.


Загрузка и настройка предобученной модели

In [16]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step 


Обучение

In [18]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.1471 - loss: 1.8534 - val_accuracy: 0.1913 - val_loss: 1.8075
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.2068 - loss: 1.8120 - val_accuracy: 0.1913 - val_loss: 1.7835
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.1809 - loss: 1.8342 - val_accuracy: 0.1913 - val_loss: 1.7837
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.1761 - loss: 1.7997 - val_accuracy: 0.1826 - val_loss: 1.7889
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.2227 - loss: 1.8061 - val_accuracy: 0.1913 - val_loss: 1.7872
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.1731 - loss: 1.8590 - val_accuracy: 0.1826 - val_loss: 1.7881
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.1737 - loss: 1.7959 - val_accuracy: 0.1913 - val_loss: 1.7885
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.1582 - loss: 1.8201 - val_accuracy: 0.2261 - val_loss:

Оценим модель

In [20]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.2f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 941ms/step - accuracy: 0.1703 - loss: 1.8017
Validation Accuracy: 0.19


In [21]:
y_true = val_generator.classes
y_pred = model.predict(val_generator)
y_pred_classes = y_pred.argmax(axis=1)

print(classification_report(y_true, y_pred_classes, target_names=val_generator.class_indices.keys()))

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step
              precision    recall  f1-score   support

      Bishop       0.00      0.00      0.00        18
        King       0.00      0.00      0.00        16
      Knight       0.00      0.00      0.00        22
        Pawn       0.19      1.00      0.32        22
       Queen       0.00      0.00      0.00        16
        Rook       0.00      0.00      0.00        21

    accuracy                           0.19       115
   macro avg       0.03      0.17      0.05       115
weighted avg       0.04      0.19      0.06       115



C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo